# Load survey data, inspect, and filter by quality

We'll load all the data downloaded in [download.ipynb](download.ipynb) and investigate their quality, what is available, etc. We'll also convert the raw data from the [MAG88T](https://www.ngdc.noaa.gov/mgg/dat/geodas/docs/mag88t.pdf) format into CSV for easier loading with pandas.

In [1]:
import zipfile
from pathlib import Path
import pandas as pd
import numpy as np
import pygmt as pg
import matplotlib.pyplot as plt
from tqdm import tqdm

Set and create the data folders.

In [2]:
output = Path("..") / "data"
output_raw = output / "raw"
output_processed = output / "csv"
if not output_processed.exists():
    output_processed.mkdir()

Convert the zipped MAG88T files to CSV for easier reading.

In [3]:
for archive_name in tqdm(list(output_raw.glob("*.zip")), ncols=100):
    with zipfile.ZipFile(archive_name) as archive:
        data_files = [
            fname for fname in archive.namelist() 
            if "mag88t" in fname and fname.endswith(".m88t")
        ]
        assert len(data_files) == 1
        data_file = data_files[0]
        with archive.open(data_file) as survey_file:
            csv_fname = Path(data_file).parts[-1].replace(".m88t", ".csv")
            with open(output_processed / csv_fname, mode="w") as csv_file:
                for line in survey_file:
                    csv_file.write(line.decode().replace("\t", ","))

100%|█████████████████████████████████████████████████████████████| 131/131 [00:34<00:00,  3.79it/s]


In [16]:
csv_files = list(sorted(output_processed.glob("*.csv")))
len(csv_files)

131

Open up one of these in pandas to see what we got.

In [17]:
pd.read_csv(csv_files[0]).dropna(axis="columns", how="all")

,SURVEY_ID,DATE,TIME,LAT,LON,ALT_BAROM,ALT_RADAR,LINEID,MAG_TOTOBS,MAG_RES
0,0530-001,19890220,115217,-17.6874,-67.0007,4417,641.0,6030,24593.12,24372.82
1,0530-001,19890220,115217,-17.6871,-67.0011,4417,644.0,6030,24593.18,24372.78
2,0530-001,19890220,115218,-17.6869,-67.0014,4417,648.0,6030,24593.18,24372.72
3,0530-001,19890220,115218,-17.6867,-67.0018,4417,649.0,6030,24593.22,24372.66
4,0530-001,19890220,115219,-17.6864,-67.0021,4418,651.0,6030,24593.26,24372.60
...,...,...,...,...,...,...,...,...,...,...
712388,0530-001,19891230,161628,-20.9426,-67.5328,4445,940.0,518,24314.30,24377.40
712389,0530-001,19891230,161629,-20.9430,-67.5328,4442,939.0,518,24315.60,24378.74
712390,0530-001,19891230,161629,-20.9435,-67.5328,4441,939.0,518,24316.86,24380.06
712391,0530-001,19891230,161629,-20.9439,-67.5328,4442,939.0,518,24318.10,24381.34


In [18]:
pd.read_csv(csv_files[-10]).dropna(axis="columns", how="all")

,SURVEY_ID,DATE,TIME,LAT,LON,ALT_RADAR,LINEID,MAG_TOTOBS,MAG_DECLIN,MAG_HORIZ,MAG_X_NRTH,MAG_Y_EAST,MAG_Z_VERT,MAG_INCLIN
0,PM-WORLD_53-70_64A,19640120,211500,-35.167,-58.383,128,223,25400.0,-0.917,21460.0,21457.0,-343.0,13570.0,-32.300
1,PM-WORLD_53-70_64A,19640120,212000,-35.417,-58.283,128,223,25410.0,-0.867,21410.0,21408.0,-324.0,13670.0,-32.550
2,PM-WORLD_53-70_64A,19640120,212500,-35.617,-58.050,128,223,25380.0,-1.283,21280.0,21275.0,-477.0,13820.0,-33.000
3,PM-WORLD_53-70_64A,19640120,213000,-35.800,-57.767,128,223,25370.0,-1.583,21220.0,21212.0,-586.0,13890.0,-33.200
4,PM-WORLD_53-70_64A,19640120,213500,-35.950,-57.517,128,223,25400.0,-1.383,21160.0,21154.0,-511.0,14040.0,-33.567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858,PM-WORLD_53-70_64A,19690502,120000,-23.633,-48.150,216,T138,23798.0,-14.250,21854.0,21182.0,-5380.0,-9420.0,-23.317
5859,PM-WORLD_53-70_64A,19690502,120500,-23.567,-47.833,216,T138,23832.0,-14.717,21888.0,21170.0,-5561.0,-9427.0,-23.300
5860,PM-WORLD_53-70_64A,19690502,121000,-23.500,-47.500,216,T138,23857.0,-14.850,21892.0,21161.0,-5611.0,-9481.0,-23.417
5861,PM-WORLD_53-70_64A,19690502,121500,-23.467,-47.167,216,T138,23922.0,-15.167,21891.0,21128.0,-5727.0,-9647.0,-23.783


It seems that the amount of data available vary quite a bit. Some data don't seem to have proper altitude measurements. Others don't include diurnal corrections, which could be very hard to redo (we could search for a close geomagnetic observatory with available data at the time of acquisition).

Let's try to figure out how many surveys actually have each of these fields. First, create a DataFrame that indicates what each survey has.

In [51]:
info = dict(
    has_qualco=[],
    has_igrf=[],
    has_gps=[],
    has_alt=[],
    has_rawmag=[],
    has_cormag=[],
    has_magdiurnal=[],
    has_resmag=[],
    has_magdec=[],
    has_maginc=[],
    name=[f.name for f in csv_files],
)
for fname in tqdm(csv_files, ncols=100):
    data = pd.read_csv(fname, low_memory=False).dropna(axis="columns", how="all")
    info["has_qualco"].append("MAG_QUALCO" in data)
    info["has_igrf"].append("IGRF_CORR" in data)
    info["has_gps"].append("ALT_GPS" in data)
    info["has_alt"].append("ALT_GPS" in data or "ALT_BAROM" in data)
    info["has_rawmag"].append("MAG_TOTOBS" in data)
    info["has_cormag"].append("MAG_TOTCOR" in data)
    info["has_magdiurnal"].append("MAG_DICORR" in data)
    info["has_resmag"].append("MAG_RES" in data)
    info["has_magdec"].append("MAG_DECLIN" in data)
    info["has_maginc"].append("MAG_INCLIN" in data)
survey_info = pd.DataFrame(info)
survey_info

100%|█████████████████████████████████████████████████████████████| 131/131 [01:23<00:00,  1.56it/s]


,has_qualco,has_igrf,has_gps,has_alt,has_rawmag,has_cormag,has_magdiurnal,has_resmag,has_magdec,has_maginc,name
0,False,False,False,True,True,False,False,True,False,False,0530-001.csv
1,False,False,False,True,True,True,False,False,False,False,0530-002.csv
2,False,False,False,True,True,True,False,False,False,False,0530-003.csv
3,False,False,False,True,True,True,False,False,False,False,0530-004.csv
4,False,False,False,True,True,True,False,True,False,False,admap_bas_post1990.csv
...,...,...,...,...,...,...,...,...,...,...,...
126,False,False,False,False,True,False,False,False,True,True,pm-world_53-70_66.csv
127,False,False,False,False,True,False,False,False,True,True,pm-world_53-70_69.csv
128,False,False,False,False,True,False,False,False,True,True,pm-world_53-70_70.csv
129,False,True,False,False,True,False,False,True,False,False,whoi_arctc_ocn.csv


Now count how many surveys have each field.

In [52]:
survey_info.drop("name", axis="columns").sum(axis="rows")

has_qualco          0
has_igrf           15
has_gps             1
has_alt            68
has_rawmag        124
has_cormag         18
has_magdiurnal     13
has_resmag         66
has_magdec         59
has_maginc         59
dtype: int64

So not many have altitude information. Most have the total observed field with no diurnal correction or IGRF calculated field.

Only 1 survey has altitude from GPS, which indicates that most of this data should **not be used for quantitative studies** like inversions and modelling. But we can still make use of some of it for tutorials and classes where accuracy is not paramount.

Let's see how many have altitude and the total field anomaly that are usable.

In [53]:
survey_ok = survey_info.name[survey_info.has_alt & survey_info.has_resmag]
survey_ok

0                      0530-001.csv
4            admap_bas_post1990.csv
5             admap_bas_pre1990.csv
6                admap_behrendt.csv
7                 admap_casertz.csv
8                   admap_chile.csv
9                 admap_ganovex.csv
10                 admap_gitara.csv
11               admap_rus_east.csv
12               admap_rus_west.csv
15                   admap_usac.csv
17                afghan_mag06a.csv
18                afghan_mag06b.csv
19                afghan_mag06c.csv
20                afghan_mag06d.csv
21                afghan_mag06e.csv
22                afghan_mag06f.csv
23                afghan_mag06g.csv
24                afghan_mag06h.csv
25                afghan_mag08a.csv
40     norda-alpha-r_canada-b77.csv
41     norda-alpha-r_canada-b78.csv
47              nrl-arctic-1998.csv
48              nrl-arctic-1999.csv
59            nrl_lincoln-sea75.csv
83      pm-hi-alt-world-nc-121k.csv
84       pm-hi-alt-world-nc-54r.csv
85        pm-hi-alt-world-p2

And any that have mostly good information.

In [55]:
survey_good = survey_info.name[survey_info.has_alt & survey_info.has_magdec & survey_info.has_maginc]
survey_good

26      can-scan_bc_ne-pacific.csv
27    can-scan_canadian-arctic.csv
28     can-scan_central-canada.csv
29     can-scan_eastern-canada.csv
30          can-scan_greenland.csv
31            can-scan_iceland.csv
32             can-scan_nordic.csv
33     can-scan_western-canada.csv
66     pm-hi-alt-world-c32-051.csv
67     pm-hi-alt-world-c32-052.csv
68     pm-hi-alt-world-c32-053.csv
69     pm-hi-alt-world-c32-054.csv
70     pm-hi-alt-world-c32-753.csv
71     pm-hi-alt-world-c32-754.csv
72     pm-hi-alt-world-c32-851.csv
73     pm-hi-alt-world-c32-852.csv
74     pm-hi-alt-world-c32-853.csv
75     pm-hi-alt-world-c32-854.csv
76     pm-hi-alt-world-c32-951.csv
77     pm-hi-alt-world-c32-953.csv
78     pm-hi-alt-world-c32-954.csv
79     pm-hi-alt-world-d32-352.csv
80     pm-hi-alt-world-d32-353.csv
81     pm-hi-alt-world-d32-451.csv
82     pm-hi-alt-world-d32-452.csv
89     pm-juan-de-fuca-a32-154.csv
93     pm-samoa-c32-651flt9456.csv
94     pm-samoa-c32-751flt9467.csv
95    pm-samoa-c32-8